In [2]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import timm

# 데이터셋 디렉토리 경로
data_dir = "C:/Users/USER/Desktop/Datas/Low Quality/f2f_data"

# 전처리를 위한 변환 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 훈련 데이터셋
train_dataset = ImageFolder(root=data_dir + "/train", transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 검증 데이터셋
val_dataset = ImageFolder(root=data_dir + "/val", transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 테스트 데이터셋
test_dataset = ImageFolder(root=data_dir + "/test", transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 모델 정의 (여기서는 EfficientNetB0 모델 사용)
model = timm.create_model("tf_efficientnet_b0", pretrained=True)
num_classes = len(train_dataset.classes)

# EfficientNet의 마지막 레이어를 변경
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, num_classes)

# GPU를 사용할 경우
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 옵티마이저 정의
optimizer = optim.Adam(model.parameters(), lr=0.001)


# 훈련 및 검증 함수
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    accuracy = 100 * correct / total

    return epoch_loss, accuracy

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    accuracy = 100 * correct / total

    return epoch_loss, accuracy

# 모델 훈련
num_epochs = 10
best_val_accuracy = 0.0

for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, train_dataloader, criterion, optimizer, device)
    val_loss, val_accuracy = evaluate(model, val_dataloader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%")
    print("-" * 50)

    # 검증 정확도가 최고값인 경우 모델 저장
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), "best_model.pt")

# 테스트 데이터셋으로 모델 평가
model.load_state_dict(torch.load("best_model.pt"))
test_loss, test_accuracy = evaluate(model, test_dataloader, criterion, device)
print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/10
Train Loss: 0.1598 | Train Accuracy: 92.67%
Val Loss: 0.1827 | Val Accuracy: 94.78%
--------------------------------------------------
Epoch 2/10
Train Loss: 0.0481 | Train Accuracy: 97.83%
Val Loss: 0.0104 | Val Accuracy: 99.75%
--------------------------------------------------
Epoch 3/10
Train Loss: 0.0636 | Train Accuracy: 98.00%
Val Loss: 0.1305 | Val Accuracy: 94.28%
--------------------------------------------------
Epoch 4/10
Train Loss: 0.0199 | Train Accuracy: 99.58%
Val Loss: 0.0154 | Val Accuracy: 99.75%
--------------------------------------------------
Epoch 5/10
Train Loss: 0.0062 | Train Accuracy: 99.92%
Val Loss: 0.0217 | Val Accuracy: 99.00%
--------------------------------------------------
Epoch 6/10
Train Loss: 0.0034 | Train Accuracy: 99.83%
Val Loss: 0.0238 | Val Accuracy: 99.00%
--------------------------------------------------
Epoch 7/10
Train Loss: 0.0084 | Train Accuracy: 99.67%
Val Loss: 0.0249 | Val Accuracy: 99.50%
-----------------------------